<a href="https://colab.research.google.com/github/Kushal94gitgit/Research-project/blob/main/inversecode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import joblib, json

fwd_model = keras.models.load_model(
    "bandgap_mlp_keras.h5",
    compile=False
)

scaler_X = joblib.load("scaler_X.pkl")

with open("feature_names.json", "r") as f:
    feature_names = json.load(f)

n_features = len(feature_names)


In [3]:
fwd_model.trainable = False


In [4]:
def build_inverse_nn(n_features):
    inp = keras.Input(shape=(1,))
    x = layers.Dense(64, activation="relu")(inp)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dense(256, activation="relu")(x)
    out = layers.Dense(n_features, activation="linear")(x)
    return keras.Model(inp, out)

inv_model = build_inverse_nn(n_features)


In [5]:
target_bg = keras.Input(shape=(1,))

# wrap inverse model
x_scaled = keras.layers.Lambda(
    lambda x: inv_model(x),
    name="inv_wrapper"
)(target_bg)

# project 17 -> 24
x_scaled = keras.layers.Dense(24, name="projection_17_to_24")(x_scaled)

# wrap forward model
bg_pred = keras.layers.Lambda(
    lambda x: fwd_model(x),
    name="fwd_wrapper"
)(x_scaled)

inverse_pipeline = keras.Model(
    inputs=target_bg,
    outputs=bg_pred,
    name="inverse_pipeline"
)

inverse_pipeline.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="mse"
)


In [6]:
target_bandgaps = np.linspace(0.5, 5.5, 200).reshape(-1, 1)

inverse_pipeline.fit(
    target_bandgaps,
    target_bandgaps,
    epochs=300,
    batch_size=32,
    verbose=2
)


Epoch 1/300
7/7 - 1s - 159ms/step - loss: 8.1926
Epoch 2/300
7/7 - 0s - 11ms/step - loss: 8.0515
Epoch 3/300
7/7 - 0s - 20ms/step - loss: 7.4588
Epoch 4/300
7/7 - 0s - 21ms/step - loss: 7.3577
Epoch 5/300
7/7 - 0s - 12ms/step - loss: 6.9173
Epoch 6/300
7/7 - 0s - 13ms/step - loss: 6.8298
Epoch 7/300
7/7 - 0s - 11ms/step - loss: 6.6457
Epoch 8/300
7/7 - 0s - 20ms/step - loss: 6.2712
Epoch 9/300
7/7 - 0s - 11ms/step - loss: 5.9632
Epoch 10/300
7/7 - 0s - 21ms/step - loss: 5.6388
Epoch 11/300
7/7 - 0s - 11ms/step - loss: 5.1296
Epoch 12/300
7/7 - 0s - 11ms/step - loss: 4.7110
Epoch 13/300
7/7 - 0s - 11ms/step - loss: 4.0018
Epoch 14/300
7/7 - 0s - 11ms/step - loss: 2.9370
Epoch 15/300
7/7 - 0s - 21ms/step - loss: 2.2488
Epoch 16/300
7/7 - 0s - 22ms/step - loss: 1.3045
Epoch 17/300
7/7 - 0s - 12ms/step - loss: 0.6612
Epoch 18/300
7/7 - 0s - 21ms/step - loss: 0.3946
Epoch 19/300
7/7 - 0s - 13ms/step - loss: 0.2385
Epoch 20/300
7/7 - 0s - 13ms/step - loss: 0.2467
Epoch 21/300
7/7 - 0s - 12ms

In [11]:
target_bg = keras.Input(shape=(1,))

x_scaled_17 = inv_model(target_bg)   # ← REQUIRED

projection = keras.layers.Dense(24, name="projection_17_to_24")
x_scaled_24 = projection(x_scaled_17)


In [12]:
# define once
projection = keras.layers.Dense(24, name="projection_17_to_24")

# use everywhere
x_scaled_24 = projection(x_scaled_17)


In [13]:
bg_request = np.array([[2.0]])  # desired bandgap in eV

# inverse model output (17)
x_scaled_17 = inv_model.predict(bg_request)

# project 17 -> 24 (use the SAME Dense layer weights if trained, or define once)
projection = keras.layers.Dense(24)
x_scaled_24 = projection(x_scaled_17)

# now inverse scale
x_pred = scaler_X.inverse_transform(x_scaled_24)

for name, val in zip(feature_names, x_pred[0]):
    print(name, val)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
Sr.No. 19.44709826171821
material_id 24.12443462529939
formula_pretty 2.3920274521140255
band_gap 4.439739791548203
energy_above_hull 1.4007843614670692
formation_energy_per_atom 0.032827320698552986
predicted_formation_energy_per_atom -1.475575102370938
density 0.055666208942273636
volume 6.540734760528343
nsites 87.83745281329107
Elements 8.080226020080705
lattice_a 86.51088292560439
lattice_b 9.911543653298175
lattice_c 86.7160009392348
lattice_alpha 47.44849229176524
lattice_beta 43.69029913675039
lattice_gamma 10.45660670326857
